In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

# Load dataset

In [7]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'control': 0, '100nM': 1}

# Load graph dataset and process if neede
graph_path = data_dir / '13cyc' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset(2230):
Number of graphs: 2230
Number of features: 13
Number of classes: 2
Train set: 1071, test set: 892, val set: 267
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 16106], pos=[2827, 2], labels=[2827, 13], nuclei=[2827], weight=[16106], condition=[32], fov=[32], id=[32], train_mask=[2827], test_mask=[2827], edge_attr=[16106, 2], x=[2827, 13], y=[32], edge_weight=[16106], name=[32], batch=[2827], ptr=[33])



In [9]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset(2230):
Number of graphs: 2230
Number of features: 13
Number of classes: 2
Train set: 1071, test set: 892, val set: 267
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 18032], pos=[3150, 2], labels=[3150, 13], nuclei=[3150], weight=[18032], condition=[32], fov=[32], id=[32], train_mask=[3150], test_mask=[3150], edge_attr=[18032, 2], x=[3150, 13], y=[32], edge_weight=[18032], name=[32], batch=[3150], ptr=[33])



# K-fold on filter dataset

In [10]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [12]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

graph_path = data_dir / '13cyc' / 'graphs' 
dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [13]:
len(dataset_filtered)

2117

In [14]:
dataset_name = '13PPI'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / '13PPI' /"saved_models" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_10152023_{dataset_name}_{condition}'

In [15]:
AVAIL_GPUS = [1]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
pools = ['mean', 'max', 'sum', 'attention', 'attention2']
num_layers = [2,3,4]
hiddens = [16, 32, 64]


epochs = 80
# model = 'GAT'
model = 'GINConv'

Global seed set to 42


In [16]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, in list(itertools.product(*[num_layers, hiddens, pools])):
            # Path to the folder where the pretrained models are saved
        # CHECKPOINT_PATH = checkpoint_folder / f'GAT_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH = checkpoint_folder / f'GIN_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=13,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                embedding=False,
                                                batch_size=128,
                                                graph_pooling=pool)
        run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


GINConv


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | GIN              | 1.1 K 
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shu

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇████████████▇██████
train_auc,▁▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████▇██████
train_f1,▁▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇████████████▇▇█████
train_loss_epoch,█▅▄▄▄▄▃▃▃▃▂▃▃▃▂▃▂▂▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▂▂▂▂▂▂▁
train_loss_step,█▇▆▆▅▅▅▇▅▄▅▅▆▅▄▄▆▅▆▄▆▄▂▅▅▃▄▃▃▄▅▅▆▃▄▅▄▃▄▁
trainer/global_step,▁▁▁▁▁▁▁▂▂▃▃▃▂▂▂▂▂▄▄▄▅▅▂▂▂▂▂▆▆▆▆▆▃▃▃▃▃███
val_acc,▁▃▄▃▂▄▄▆█▆▇▂▄▆▆▄▇▅▆▆▆▇▅▅▆▅▅▆▅▅▄▆█▅▆▅▆▆▅▅
val_auc,▁▄▅▆▆▆▆▇█▇█▅▇▇▇▇█▅██▇█▇▇██▆▇▇▇▅▇▇▇▇▇▆▇▇▆
val_f1,▁▃▄▂▂▃▅▆█▇▆▃▄▆▇▃▇▇▆▆▆▇▆▅▆▇▆▇▅▅▆▇▇▅▆▆▆▆▅▆


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▄▄▄▄▅▅▅▆▅▅▅▅▆▆▆▆▆▇▆▆▇▇▆▆▇▇▆▇▇▇▇▇▇▆███▇
train_auc,▁▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇
train_f1,▁▃▄▄▄▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▆▆▇▇▆▇▇▇▇▇▇▆███▇
train_loss_epoch,█▆▆▅▅▅▄▄▄▃▄▄▄▄▃▃▃▄▃▃▄▃▃▃▂▃▃▃▃▂▂▂▂▂▂▃▂▁▁▂
train_loss_step,▅▅▄▄▅▅▄▅▄▄▆▄█▃▃▄▃▆▄▄▅▄█▄▃▂▄▇▃▃▂▃▂▂▂▁▃▂▁▅
trainer/global_step,▁▁▁▁▁▁▁▂▂▃▃▃▂▂▂▂▂▄▄▄▅▅▂▂▂▂▂▆▆▆▆▆▃▃▃▃▃███
val_acc,▅▂▁▆▅▆▅▅▇▇▅▅▃▅▆█▆▆▇▆▃▇▇▆▇▇▂▇▅▆▇▇▇▆▄▇▆▇▆▅
val_auc,▁▄▄▅▅▆▃▆█▇▄▇▇▇▆▇▇▇▇▆▅▇▆▅█▇▁▄▆▇▇▅▆▆▆▇▆▇▅▇
val_f1,▇▂▁▇▅▇▆▅█▇▆▇▇▆▇█▇▅█▇▃██▇▇▇▃█▅▆█▇█▅▄█▇▇▇▅


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model       | GIN              | 30.9 K
2  | head        | Sequential       | 2.1 K 
3  | loss_module | CrossEntropyLoss | 0     
4  | train_acc   | BinaryAccuracy   | 0     
5  | train_auroc | BinaryAUROC      | 0     
6  | train_f1    | BinaryF1Score    | 0     
7  | valid_acc   | BinaryAccuracy   | 0     
8  | valid_auroc | BinaryAUROC      | 0     
9  | valid_f1    | BinaryF1Score    | 0     
10 | pool        | Attention_module | 4.2 K 
--------------------------------------------------
37.3 K    Trainable params
0         Non-trainable params
37.3 K    Total params
0.149     Total estimated model para

GINConv


`Trainer.fit` stopped: `max_epochs=80` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▄▄▄▄▅▅▅▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▇▆▇▇▆▇▇▇▆▇▇▇▇▇█
train_auc,▁▄▄▄▅▅▆▆▆▆▆▆▆▆▇▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████
train_f1,▁▃▄▄▄▄▅▅▅▄▅▅▅▅▆▆▆▆▆▆▅▆▆▆▆▆▆▇▇▆▇▇▇▆▇▇▇▇▇█
train_loss_epoch,█▆▆▅▅▅▄▄▅▄▄▄▄▄▃▄▃▃▃▃▄▃▃▃▃▂▃▂▂▃▂▂▃▂▂▂▁▂▂▁
train_loss_step,▅▅▅▅▃▃▃▃▃▃▃▅█▃▂▄▄▂▄▄▄▃▃▃▄▃▃▃▃▃▃▃▃▂▃▂▁▂▄▃
trainer/global_step,▁▁▁▁▁▁▁▂▂▃▃▃▂▂▂▂▂▄▄▄▅▅▂▂▂▂▂▆▆▆▆▆▃▃▃▃▃███
val_acc,▅▅▅▆▇▇█▇█▆▇▇▆▆▆▆▆██▅▅▇▇▅█▇▅▇▇▇▄▅▄▁▄▆▅▅▆▆
val_auc,▃▄▅▅▇▇▇▆█▆▇▇▇▇▆▆▇▇▇▄█▇▇▇▇█▆▇██▁▃▆▆▅▇▅▇▆▄
val_f1,▇▆▆▇▇▇█▇▇▇▇█▇▇▇▆▆█▇▇▅▇▇▅█▇▅▆▇▇▅▆▅▁▇▅▆▅▇▇
